In [27]:
# Import dependency 
import numpy as np 
import torch 
import torch.optim as optim
import torch.nn as nn 

In [28]:
# Data generation 
true_b = 1
true_w = 2 
N = 100

# set the random seed for numpy 
np.random.seed(43)

x= np.random.rand(N,1)
epsilon = (.1 * np.random.rand(N,1))

y = true_b + true_w *x + epsilon

In [29]:
#Generate training and validating sets
idx = np.arange(N)

# Use first 80 random indices for train 
train_idx = idx[:int(N*.8)]
val_idx = idx[int(N*.8):]

# Generate train and validation sets
x_train, y_train = x[train_idx], y[train_idx]


In [30]:
# Data preparation 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Transform data from numpy array to torch tensor
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

In [38]:
# Build a Dataset
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])
print(len(train_data))

(tensor([0.1151]), tensor([1.2404]))
80


In [39]:
from torch.utils.data import DataLoader

# Build a data loader that yields mini-batches of size 2
train_loader = DataLoader(
        dataset=train_data,
        batch_size=2,
        shuffle=True
)
print(train_loader)

In [40]:
print(next(iter(train_loader)))

[tensor([[0.5253],
        [0.1445]]), tensor([[2.1028],
        [1.3777]])]


In [41]:
#Define the train step

def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop 
    def perform_train_step(x,y):
        
        # Set the model to TRAIN mode
        model.train()
        
        # Step1: Compute the model's predicition - forward pass
        yhat = model(x)
        
        # Step2: Compute the loss
        loss = loss_fn(yhat, y)
        
        # Step3: Compute gradients for "b" and "w" parameters
        loss.backward()
        
        # Step4: Updates parameters using gradients and the learning rate
        optimizer.step()
        optimizer.zero_grad()
        
        #print(model.state_dict())
        # Return the loss 
        return loss.item()
    #Return the function that will be called inside the train loop
    return perform_train_step

In [42]:
# Model config 

# Define the model 
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    def forward(self,x):
        return self.linear(x)


# Set learning rate 
lr = 0.1

torch.manual_seed(42)

# Create a model and send it to the device 
model = ManualLinearRegression().to(device)
print(model.state_dict())

# Define a SGD optimizer to update the parameters 
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# Define a MSE loss function 
loss_fn = nn.MSELoss(reduction="mean")


# Create a train_step 
train_step = make_train_step(model, loss_fn, optimizer)

OrderedDict([('linear.weight', tensor([[0.7645]])), ('linear.bias', tensor([0.8300]))])


In [43]:
# Model training

n_epochs = 1000
losses = []
for epoch in range(n_epochs):
    
    # Add the inner loop to fetch the data
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        #The dataset "lives" in the CPU, so do our mini-batches
        # we need send to mini-batches to the device 
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Performs one train step and returns the corresonding loss 
        mini_batch_loss = train_step(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
        
        
    # Compute average loss over all mini-batches
    loss = np.mean(mini_batch_loss)
    losses.append(loss)
   

In [44]:
# Model's parameter before training 
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[2.0054]])), ('linear.bias', tensor([1.0554]))])


In [30]:
# Check model's parameters
#print(model.state_dict())

In [31]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([1.0510], requires_grad=True)]
